# Practical Lesson 6: Bin Packing

### __Problem Description__:

A college wants to schedule exams for 6 subjects. There is a rule that prohibits any student from having to take more than one exam per day. The table below shows which students (identified by numbers) will take the exam for each subject. For example, A and B cannot be scheduled on the same day because of student 1. However, B and C could be scheduled on the same day, since no student will take both exams.

| Subject | Students |
|:---:|:---:|
| A | 1, 2, 3, 4 |
| B | 1, 5 |
| C | 3, 6, 7 |
| D | 5, 7 |
| E | 2, 7 |
| F | 4, 5 |

Create an ILP model to schedule these exams in the minimum number of days possible.
**Hint: base your model on the bin packing problem.**

In [10]:
from mip import *

def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value}\n")

    print("Solution:")
    for v in model.vars:
      if v.x > 0.00001 and v.name.find("x")!=-1:
        print(v.name, " = ", v.x)

def save(model, filename):
    model.write(filename) 
    with open(filename, "r") as f: 
        print(f.read())

In [11]:
# First of all i run lesson6_patterns_gen.cpp to get the possible patterns

# If bit i is set in pattern p, it means subject i is selected
patterns = [
    0b000000,
    0b100000,
    0b010000,
    0b110000,
    0b001000,
    0b000100,
    0b100100,
    0b000010,
    0b010010,
    0b000110,
    0b000001,
    0b001001
]

qtd_subjects = 6


In [12]:
model = Model(name="Bin Packing", sense=MINIMIZE, solver_name=CBC)

# x[i] : 1 if the i-th pattern is selected
x = [model.add_var(name=f"x_{i}", var_type=BINARY) for i in range(len(patterns))]

# Objective function
model.objective = xsum(x[i] for i in range(len(patterns)))

# Subjects constraints
for j in range(qtd_subjects):
    model.add_constr(name=f"subject_{j}_constraint", lin_expr=xsum(x[i] * ((patterns[i] & (1 << j)) > 0) for i in range(len(patterns))) == 1)

save(model, "../data/lesson6.lp")


\Problem name: Bin Packing

Minimize
OBJROW: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9
 + x_10 + x_11
Subject To
subject_0_constraint:  x_10 + x_11 = 1
subject_1_constraint:  x_7 + x_8 + x_9 = 1
subject_2_constraint:  x_5 + x_6 + x_9 = 1
subject_3_constraint:  x_4 + x_11 = 1
subject_4_constraint:  x_2 + x_3 + x_8 = 1
subject_5_constraint:  x_1 + x_3 + x_6 = 1
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1
 0 <= x_11 <= 1
Integers
x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 
x_10 x_11 
End



In [13]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3.0

Solution:
x_6  =  1.0
x_8  =  1.0
x_11  =  1.0
